In [1]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import xml.etree.ElementTree as ET
from lxml import etree
from scipy.sparse import hstack
import numpy as np
import warnings
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize

/home/sirius/anaconda3/envs/nelp_dl/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [43]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

#Spacy
import spacy
nlp = spacy.load('en_core_web_sm')

# Other
import re
import json
import string
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

[nltk_data] Downloading package stopwords to /home/sirius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [29]:
def parse_sentence_level(path):
    tree = ET.parse(path)
    root = tree.getroot()
    print(root)
    sent_id = []
    sent_text = []
    opinion_target = []
    opinion_category = []
    opinion_polarity = []
    for review in root.findall('Review'):
        for sent in review.findall('./sentences/sentence'):
            sent_id.append(sent.get('id'))
            sent_text.append(sent.find('text').text)
            target = ""
            polarity = ""
            category = ""
            for opinion in sent.findall('./Opinions/Opinion'):
                target += " " + opinion.get('target')
                polarity += " " + opinion.get('polarity')
                category += " " + opinion.get('category')
            opinion_target.append(target)
            opinion_category.append(category)
            opinion_polarity.append(polarity)
    return sent_id, sent_text, opinion_target, opinion_category, opinion_polarity

In [2]:
#xml parser
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [23]:
parse_sentence_level("ABSA16_Restaurants_Train_SB1_v2.xml")

<Element 'Reviews' at 0x7fd261a60f50>
1004293:0 Judging from previous posts this used to be a good place, but not any longer.
1004293:1 We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.
1004293:2 They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.
1004293:3 The food was lousy - too sweet or too salty and the portions tiny.
1004293:4 After all that, they complained to me about the small tip.
1004293:5 Avoid this place!
1014458:0 I have eaten at Saul, many times, the food is always consistently, outrageously good.
1014458:1 Saul is the best restaurant on Smith Street and in Brooklyn.
1014458:2 The duck confit is always amazing and the foie gras terrine with figs was out of this world.
1014458:3 The wine list is interesting and has many good values.
1014458:4 For the price, you cannot eat this well in Manhattan.
1016296:0 I was very d

493013:2 I LOVE their Thai
493013:3 noodles with shrimp and chicken and coconut juice is the MUST!
493013:4 I was in love with Pongsri on 48th, but compared to Suan it is slow in service and overpriced.
493013:5 I will go back to Suan soon!
493013:6 In summer-eat outside on a terrace (another great feature of Suan)!!!
493514:0 I cannot imagine a friendlier staff working in a restaurant.
493514:1 I cannot imagine better Indian food in all of the city.
493514:2 I cannot imagine you not rushing out to eat there.
494850:0 This restaurant used to be our regular Thursday night dinner location.
494850:1 During the course of the past 3 months, the chef and staff changed and it was not for the better.
494850:2 I wish they would change back to what it was before.
494850:3 The food now is inconsistent.
496132:0 This is the kind of place you'd like to take all your friends to and still keep a secret.
496132:1 The setting is casual and romantic.
496132:2 Prices are very good.
496132:3 The food is e

(['1004293:0',
  '1004293:1',
  '1004293:2',
  '1004293:3',
  '1004293:4',
  '1004293:5',
  '1014458:0',
  '1014458:1',
  '1014458:2',
  '1014458:3',
  '1014458:4',
  '1016296:0',
  '1016296:1',
  '1016296:2',
  '1016296:3',
  '1016296:4',
  '1016296:5',
  '1028246:0',
  '1028246:1',
  '1028246:2',
  '1032695:0',
  '1032695:1',
  '1032695:2',
  '1032695:3',
  '1041457:0',
  '1041457:1',
  '1041457:2',
  '1041457:3',
  '1041457:4',
  '1041457:5',
  '1041457:6',
  '1053543:0',
  '1053543:1',
  '1053543:2',
  '1053543:3',
  '1053543:4',
  '1055910:0',
  '1055910:1',
  '1055910:2',
  '1055910:3',
  '1055910:4',
  '1055910:5',
  '1058221:0',
  '1058221:1',
  '1058221:2',
  '1058221:3',
  '1058221:4',
  '1058221:5',
  '1058221:6',
  '1058221:7',
  '1062641:0',
  '1062641:1',
  '1062641:2',
  '1062641:3',
  '1062641:4',
  '1064477:0',
  '1064477:1',
  '1064477:2',
  '1064477:3',
  '1064477:4',
  '1074868:0',
  '1074868:1',
  '1074868:2',
  '1074868:3',
  '1074868:4',
  '1074868:5',
  '1084394

In [30]:
train_sent_id, train_text,train_opinion_target,train_opinion_category, train_opinion_polarity  = parse_sentence_level("ABSA16_Restaurants_Train_SB1_v2.xml")

<Element 'Reviews' at 0x7fd260f9b7d0>


In [38]:
reviews_train = pd.DataFrame([train_sent_id, train_text,train_opinion_target,train_opinion_category, train_opinion_polarity],
            index=['sentence_id','text','aspect_target','aspect_category','polarity']).T

In [39]:
print(reviews_train.groupby('aspect_category').size().sort_values(ascending=False))

#how many categories
print("number of categories",reviews_train.aspect_category.nunique())

aspect_category
 RESTAURANT#GENERAL                                                       342
 FOOD#QUALITY                                                             301
                                                                          292
 SERVICE#GENERAL                                                          242
 AMBIENCE#GENERAL                                                          84
                                                                         ... 
 FOOD#STYLE_OPTIONS FOOD#QUALITY RESTAURANT#GENERAL FOOD#QUALITY            1
 FOOD#STYLE_OPTIONS FOOD#QUALITY FOOD#STYLE_OPTIONS FOOD#STYLE_OPTIONS      1
 FOOD#STYLE_OPTIONS FOOD#QUALITY FOOD#STYLE_OPTIONS                         1
 FOOD#STYLE_OPTIONS FOOD#QUALITY FOOD#QUALITY                               1
 FOOD#QUALITY FOOD#STYLE_OPTIONS SERVICE#GENERAL AMBIENCE#GENERAL           1
Length: 199, dtype: int64
number of categories 199


In [40]:
reviews_train.shape

(2000, 5)

In [56]:
absa_model = Sequential()
absa_model.add(Dense(512, input_shape=(6000,), activation='relu'))
absa_model.add((Dense(256, activation='relu')))
absa_model.add((Dense(128, activation='relu')))
absa_model.add(Dense(199, activation='softmax'))
#compile model
absa_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [46]:
from keras.preprocessing.text import Tokenizer

In [47]:
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.text)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.text))

In [48]:
from sklearn.preprocessing import LabelEncoder

In [50]:
from keras.utils import to_categorical

In [71]:
label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(reviews_train.aspect_category)
dummy_category = to_categorical(integer_category)

In [68]:
#model architecture
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add((Dense(256, activation='relu')))
sentiment_model.add((Dense(128, activation='relu')))
sentiment_model.add(Dense(44, activation='softmax'))
#compile model
sentiment_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#create a word embedding of reviews data
vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(reviews_train.text)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(reviews_train.text))

#encode the label variable
label_encoder2 = LabelEncoder()
integer_sentiment = label_encoder2.fit_transform(reviews_train.polarity)
dummy_sentiment = to_categorical(integer_sentiment)

In [59]:
reviews_train.polarity.nunique()

44

In [57]:
absa_model.fit(reviews_tokenized, dummy_category, epochs=100, verbose=1)

Epoch 1/100
2000/2000 [==============================] - 2s 1ms/step - loss: 3.8338 - accuracy: 0.2050
Epoch 2/100
2000/2000 [==============================] - 1s 307us/step - loss: 2.4730 - accuracy: 0.4315
Epoch 3/100
2000/2000 [==============================] - 1s 299us/step - loss: 1.7432 - accuracy: 0.5910
Epoch 4/100
2000/2000 [==============================] - 1s 301us/step - loss: 1.2820 - accuracy: 0.7090
Epoch 5/100
2000/2000 [==============================] - 1s 298us/step - loss: 0.9682 - accuracy: 0.7895
Epoch 6/100
2000/2000 [==============================] - 1s 302us/step - loss: 0.7216 - accuracy: 0.8480
Epoch 7/100
2000/2000 [==============================] - 1s 302us/step - loss: 0.5411 - accuracy: 0.8905
Epoch 8/100
2000/2000 [==============================] - 1s 300us/step - loss: 0.3980 - accuracy: 0.9160
Epoch 9/100
2000/2000 [==============================] - 1s 300us/step - loss: 0.2718 - accuracy: 0.9380
Epoch 10/100
2000/2000 [==============================] -

2000/2000 [==============================] - 1s 304us/step - loss: 0.0023 - accuracy: 0.9990
Epoch 79/100
2000/2000 [==============================] - 1s 302us/step - loss: 0.0017 - accuracy: 0.9990
Epoch 80/100
2000/2000 [==============================] - 1s 302us/step - loss: 0.0019 - accuracy: 0.9990
Epoch 81/100
2000/2000 [==============================] - 1s 303us/step - loss: 0.0019 - accuracy: 0.9990
Epoch 82/100
2000/2000 [==============================] - 1s 302us/step - loss: 0.0016 - accuracy: 0.9995
Epoch 83/100
2000/2000 [==============================] - 1s 305us/step - loss: 0.0012 - accuracy: 0.9995
Epoch 84/100
2000/2000 [==============================] - 1s 303us/step - loss: 0.0011 - accuracy: 0.9995
Epoch 85/100
2000/2000 [==============================] - 1s 303us/step - loss: 0.0016 - accuracy: 0.9990
Epoch 86/100
2000/2000 [==============================] - 1s 304us/step - loss: 0.0017 - accuracy: 0.9990
Epoch 87/100
2000/2000 [==============================] - 1

In [69]:
sentiment_model.fit(reviews_tokenized, dummy_sentiment, epochs=100, verbose=1)

Epoch 1/100
2000/2000 [==============================] - 1s 361us/step - loss: 2.5042 - accuracy: 0.3475
Epoch 2/100
2000/2000 [==============================] - 1s 295us/step - loss: 1.3553 - accuracy: 0.5735
Epoch 3/100
2000/2000 [==============================] - 1s 300us/step - loss: 0.6678 - accuracy: 0.7995
Epoch 4/100
2000/2000 [==============================] - 1s 294us/step - loss: 0.3473 - accuracy: 0.9100
Epoch 5/100
2000/2000 [==============================] - 1s 298us/step - loss: 0.2042 - accuracy: 0.9505
Epoch 6/100
2000/2000 [==============================] - 1s 295us/step - loss: 0.1282 - accuracy: 0.9680
Epoch 7/100
2000/2000 [==============================] - 1s 297us/step - loss: 0.0810 - accuracy: 0.9820
Epoch 8/100
2000/2000 [==============================] - 1s 295us/step - loss: 0.0552 - accuracy: 0.9890
Epoch 9/100
2000/2000 [==============================] - 1s 297us/step - loss: 0.0381 - accuracy: 0.9905
Epoch 10/100
2000/2000 [==============================]

2000/2000 [==============================] - 1s 297us/step - loss: 7.6295e-06 - accuracy: 1.0000
Epoch 76/100
2000/2000 [==============================] - 1s 296us/step - loss: 7.2175e-06 - accuracy: 1.0000
Epoch 77/100
2000/2000 [==============================] - 1s 295us/step - loss: 6.9344e-06 - accuracy: 1.0000
Epoch 78/100
2000/2000 [==============================] - 1s 298us/step - loss: 6.5971e-06 - accuracy: 1.0000
Epoch 79/100
2000/2000 [==============================] - 1s 298us/step - loss: 6.3089e-06 - accuracy: 1.0000
Epoch 80/100
2000/2000 [==============================] - 1s 297us/step - loss: 6.0147e-06 - accuracy: 1.0000
Epoch 81/100
2000/2000 [==============================] - 1s 310us/step - loss: 5.7731e-06 - accuracy: 1.0000
Epoch 82/100
2000/2000 [==============================] - 1s 314us/step - loss: 5.5181e-06 - accuracy: 1.0000
Epoch 83/100
2000/2000 [==============================] - 1s 307us/step - loss: 5.3988e-06 - accuracy: 1.0000
Epoch 84/100
2000/2000 

In [62]:
test_reviews = [
    "Good, fast service.",
    "The hostess was very pleasant.",
    "The bread was stale, the salad was overpriced and empty.",
    "The food we ordered was excellent, although I wouldn't say the margaritas were anything to write home about.",
    "This place has totally weird decor, stairs going up with mirrored walls - I am surprised how no one yet broke their head or fall off the stairs"
]

In [63]:
test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))
test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))

In [72]:
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 
test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))

# Models output
test_aspect_categories = label_encoder.inverse_transform(absa_model.predict_classes(test_aspect_terms))
test_sentiment = label_encoder2.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))
for i in range(5):
    print("Review " + str(i+1) + " is expressing a  " + test_sentiment[i] + " opinion about " + test_aspect_categories[i])


Review 1 is expressing a   positive opinion about  SERVICE#GENERAL
Review 2 is expressing a   positive opinion about  SERVICE#GENERAL
Review 3 is expressing a   opinion about  FOOD#QUALITY FOOD#QUALITY
Review 4 is expressing a   positive opinion about  FOOD#QUALITY
Review 5 is expressing a   positive opinion about  AMBIENCE#GENERAL
